# Monthly Momentun Scanner Executed Weekly

In [92]:
import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
from stock_config import nifty50_stock_list as stock_list

## Input the evaluation date below (Every Friday)

In [93]:
eval_date = "2020-09-18"

#### Function to check valid trading day. If open price is not returned then next date is checked until valid date is found

In [94]:
def get_valid_date(start_date, end_date):
    valid = False
    while not valid:
        valid_check_data = yf.download('RELIANCE.NS', start = start_date, end=end_date)
        try:
            open_price = valid_check_data.iloc[0]['Open']
            valid = True
        except:
            start_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            end_date = (pd.to_datetime(end_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            valid = False
    return start_date, end_date

#### Get valid start and end dates for this year

In [95]:
start_date=eval_date
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
start_date, end_date = get_valid_date(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [96]:
start_date

'2020-09-18'

In [97]:
end_date

'2020-09-19'

### Fetch this year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [98]:
data = yf.download(stock_list, start = start_date, end=end_date)

[*********************100%***********************]  50 of 50 completed


In [99]:
this_month_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [100]:
# this_month_close

#### Calculate dates for last week

In [101]:
start_date=(pd.to_datetime(eval_date) - pd.DateOffset(months=1)).strftime('%Y-%m-%d')
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

#### Get valid start and end dates for last year

In [102]:
start_date, end_date = get_valid_date(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [103]:
start_date

'2020-08-18'

In [104]:
end_date

'2020-08-19'

### Fetch last year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [105]:
data = yf.download(stock_list, start = start_date, end=end_date)

[*********************100%***********************]  50 of 50 completed


In [106]:
last_month_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [107]:
# last_month_close

In [108]:
final_df = last_month_close.join(this_month_close)

In [109]:
final_df

,2020-08-18,2020-09-18
ADANIPORTS.NS,357.450012,357.850006
ASIANPAINT.NS,1874.099976,2028.400024
AXISBANK.NS,445.799988,443.350006
BAJAJ-AUTO.NS,3104.800049,3050.949951
BAJAJFINSV.NS,6343.049805,5888.450195
BAJFINANCE.NS,3419.899902,3474.800049
BHARTIARTL.NS,526.299988,494.549988
BPCL.NS,402.299988,412.049988
BRITANNIA.NS,3830.928467,3797.500000
CIPLA.NS,764.849976,806.250000


In [110]:
final_df.rename(columns =lambda t: t.strftime('%Y-%m-%d'), inplace=True)

## Calculate Momentum

In [111]:
last_month_date = final_df.columns[0]
this_month_date = final_df.columns[1]
final_df['Change'] = final_df[this_month_date] - final_df[last_month_date]

In [112]:
final_df['% Change'] = (final_df['Change'] / final_df[last_month_date])*100

## Final Output

In [113]:
final_df.sort_values(by=['% Change'], ascending=False, inplace=True)

In [114]:
final_df.rename_axis('MOMENTUM_ATH', inplace=True)
final_df.rename_axis('', axis='columns', inplace=True)

In [115]:
final_df.round(2).head(50)

,2020-08-18,2020-09-18,Change,% Change
MOMENTUM_ATH,,,,
ZEEL.NS,173.71,220.30,46.59,26.82
DRREDDY.NS,4520.40,5333.35,812.95,17.98
TATAMOTORS.NS,125.60,147.90,22.30,17.75
INDUSINDBK.NS,521.60,613.20,91.60,17.56
HCLTECH.NS,712.47,810.60,98.13,13.77
TECHM.NS,713.05,804.65,91.60,12.85
GRASIM.NS,663.14,745.00,81.86,12.34
WIPRO.NS,282.70,316.50,33.80,11.96
RELIANCE.NS,2118.55,2305.70,187.15,8.83


### Create an empty excel file as below filename before running below code

In [116]:
filename = f'data/output/momentum_monthlay_output.xlsx'
book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book

In [117]:
final_df.round(2).to_excel(writer, sheet_name=eval_date)
writer.save()
writer.close()

In [118]:
f"Done. Output file path and name --> {filename}"

'Done. Output file path and name --> data/output/momentum_monthly_output.xlsx'